In [1]:
import xml.etree.ElementTree as ET
from glob import glob
import pandas as pd
import numpy as np

In [2]:
#A pasta annotations contém arquivos xml para cada imagem
#Estes arquivos xml possuem os metadados da imagem, tais como o posicionamento das células
annotations = glob('../dataset/BCCD/Annotations/*.xml')


In [3]:
df = []
cnt = 0
for file in annotations:
    filename = file.split('\\')[-1]
    filename =filename.split('.')[0] + '.jpg'
    row = []
    parsedXML = ET.parse(file)
    #percorre os nós do XML para identificar os bounderies boxes de cada célula na imagem
    for node in parsedXML.getroot().iter('object'):
        blood_cells = node.find('name').text
        xmin = int(node.find('bndbox/xmin').text)
        xmax = int(node.find('bndbox/xmax').text)
        ymin = int(node.find('bndbox/ymin').text)
        ymax = int(node.find('bndbox/ymax').text)

        #cada linha representa a localização de uma célula
        row = [filename, blood_cells, xmin, xmax, ymin, ymax]
        
        #adiciona ao dataframe cada linha de célula localizada
        df.append(row)
        cnt += 1

data = pd.DataFrame(df, columns=['filename', 'cell_type', 'xmin', 'xmax', 'ymin', 'ymax'])

In [4]:
data.head()

,filename,cell_type,xmin,xmax,ymin,ymax
0,BloodImage_00000.jpg,WBC,260,491,177,376
1,BloodImage_00000.jpg,RBC,78,184,336,435
2,BloodImage_00000.jpg,RBC,63,169,237,336
3,BloodImage_00000.jpg,RBC,214,320,362,461
4,BloodImage_00000.jpg,RBC,414,506,352,445


In [5]:
#seleciona apenas células vermelhas
only_rbcs = data[data['cell_type'] == 'RBC']

In [6]:
only_rbcs.head()

,filename,cell_type,xmin,xmax,ymin,ymax
1,BloodImage_00000.jpg,RBC,78,184,336,435
2,BloodImage_00000.jpg,RBC,63,169,237,336
3,BloodImage_00000.jpg,RBC,214,320,362,461
4,BloodImage_00000.jpg,RBC,414,506,352,445
5,BloodImage_00000.jpg,RBC,555,640,356,455


In [7]:
cells_qty = only_rbcs.groupby('filename')['filename'].agg([np.count_nonzero]).rename(columns={'count_nonzero':'total'})

In [9]:
cells_qty.to_csv('quantidade_celulas_rbc.csv')

In [10]:
cells_qty['total'].sum()

4155

In [7]:
data.to_csv('localizacao_celulas.csv',index_label='Indice')